In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1023253,2021-02-22,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1023254,2021-02-23,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1023255,2021-02-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1023256,2021-02-25,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
27775,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27777,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27779,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27781,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27783,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
633998,2021-02-22,Arkansas,Arkansas,5001,1944,38.00,5000,Arkansas,AR,Arkansas,State,3017804
634000,2021-02-23,Arkansas,Arkansas,5001,1949,38.00,5000,Arkansas,AR,Arkansas,State,3017804
634002,2021-02-24,Arkansas,Arkansas,5001,1963,38.00,5000,Arkansas,AR,Arkansas,State,3017804
634004,2021-02-25,Arkansas,Arkansas,5001,1967,38.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1023253,2021-02-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1023254,2021-02-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1023255,2021-02-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1023256,2021-02-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-02-26') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
402,2021-02-26,Snohomish,Washington,30828,538.00,53061,WA,County,822083,3749.99,65.44
802,2021-02-26,Cook,Illinois,473194,9894.00,17031,IL,County,5150233,9187.82,192.11
1201,2021-02-26,Orange,California,260721,3982.00,6059,CA,County,3175692,8209.90,125.39
1599,2021-02-26,Maricopa,Arizona,509314,9061.00,4013,AZ,County,4485414,11354.89,202.01
1997,2021-02-26,Los Angeles,California,1189859,21241.00,6037,CA,County,10039107,11852.24,211.58
...,...,...,...,...,...,...,...,...,...,...,...
1022833,2021-02-26,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
1022970,2021-02-26,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00
1023076,2021-02-26,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
1023178,2021-02-26,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
402,2021-02-26,Snohomish,Washington,30828,538.00,53061,WA,County,822083,3749.99,65.44,65.44,3749.99
802,2021-02-26,Cook,Illinois,473194,9894.00,17031,IL,County,5150233,9187.82,192.11,192.11,9187.82
1201,2021-02-26,Orange,California,260721,3982.00,6059,CA,County,3175692,8209.90,125.39,125.39,8209.90
1599,2021-02-26,Maricopa,Arizona,509314,9061.00,4013,AZ,County,4485414,11354.89,202.01,202.01,11354.89
1997,2021-02-26,Los Angeles,California,1189859,21241.00,6037,CA,County,10039107,11852.24,211.58,211.58,11852.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022833,2021-02-26,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
1022970,2021-02-26,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00,0.00,4779.41
1023076,2021-02-26,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
1023178,2021-02-26,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
721183,2021-02-26,Clarke,Alabama,3431,49.00,1025,AL,County,23622,14524.60,207.43,207.43,14524.60,1
662813,2021-02-26,Hale,Alabama,2090,68.00,1065,AL,County,14651,14265.24,464.13,464.13,14265.24,2
519081,2021-02-26,Lowndes,Alabama,1307,49.00,1085,AL,County,9726,13438.21,503.80,503.80,13438.21,3
460440,2021-02-26,Etowah,Alabama,13154,319.00,1055,AL,County,102268,12862.28,311.93,311.93,12862.28,4
386586,2021-02-26,Franklin,Alabama,3988,76.00,1059,AL,County,31362,12716.03,242.33,242.33,12716.03,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973077,2021-02-26,Platte,Wyoming,589,11.00,56031,WY,County,8393,7017.75,131.06,131.06,7017.75,19
633001,2021-02-26,Sublette,Wyoming,675,7.00,56035,WY,County,9831,6866.04,71.20,71.20,6866.04,20
873836,2021-02-26,Niobrara,Wyoming,149,2.00,56027,WY,County,2356,6324.28,84.89,84.89,6324.28,21
785730,2021-02-26,Lincoln,Wyoming,1224,11.00,56023,WY,County,19830,6172.47,55.47,55.47,6172.47,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
519081,2021-02-26,Lowndes,Alabama,1307,49.00,1085,AL,County,9726,13438.21,503.80,503.80,13438.21,3,1
662813,2021-02-26,Hale,Alabama,2090,68.00,1065,AL,County,14651,14265.24,464.13,464.13,14265.24,2,2
459084,2021-02-26,Clay,Alabama,1434,54.00,1027,AL,County,13235,10834.91,408.01,408.01,10834.91,22,3
221323,2021-02-26,Walker,Alabama,6509,251.00,1127,AL,County,63521,10247.00,395.14,395.14,10247.00,32,4
559638,2021-02-26,Greene,Alabama,871,32.00,1063,AL,County,8111,10738.50,394.53,394.53,10738.50,24,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558289,2021-02-26,Hot Springs,Wyoming,360,3.00,56017,WY,County,4413,8157.72,67.98,67.98,8157.72,17,19
766260,2021-02-26,Uinta,Wyoming,2054,12.00,56041,WY,County,20226,10155.25,59.33,59.33,10155.25,4,20
785730,2021-02-26,Lincoln,Wyoming,1224,11.00,56023,WY,County,19830,6172.47,55.47,55.47,6172.47,22,21
220288,2021-02-26,Teton,Wyoming,3348,9.00,56039,WY,County,23464,14268.67,38.36,38.36,14268.67,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.44,3749.99,14,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.44,3749.99,14,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.44,3749.99,14,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.44,3749.99,14,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.44,3749.99,14,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020838,2021-02-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1020839,2021-02-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1020840,2021-02-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1020841,2021-02-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
718437,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,207.43,14524.60,47,1,2.00
718438,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,207.43,14524.60,47,1,0.00
718439,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,207.43,14524.60,47,1,1.00
718440,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,207.43,14524.60,47,1,4.00
718441,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,207.43,14524.60,47,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798774,2021-02-22,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
798775,2021-02-23,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
798776,2021-02-24,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
798777,2021-02-25,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
517001,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,503.80,13438.21,1,3,1.00,0.00
517002,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,503.80,13438.21,1,3,0.00,0.00
517003,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,503.80,13438.21,1,3,0.00,0.00
517004,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,503.80,13438.21,1,3,0.00,0.00
517005,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,503.80,13438.21,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515306,2021-02-22,Albany,Wyoming,3893,11.00,56001,WY,County,38880,10012.86,28.29,28.29,10033.44,23,6,6.00,0.00
515307,2021-02-23,Albany,Wyoming,3892,11.00,56001,WY,County,38880,10010.29,28.29,28.29,10033.44,23,6,-1.00,0.00
515308,2021-02-24,Albany,Wyoming,3893,11.00,56001,WY,County,38880,10012.86,28.29,28.29,10033.44,23,6,1.00,0.00
515309,2021-02-25,Albany,Wyoming,3894,11.00,56001,WY,County,38880,10015.43,28.29,28.29,10033.44,23,6,1.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-02-26') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
264071,2021-02-26,Imperial,California,26855,586.00,6025,CA,County,181215,14819.41,323.37,323.37,14819.41,1,2,12.00,1.00
1997,2021-02-26,Los Angeles,California,1189859,21241.00,6037,CA,County,10039107,11852.24,211.58,211.58,11852.24,2,5,1758.00,139.00
461748,2021-02-26,Inyo,California,1264,36.00,6027,CA,County,18039,7007.04,199.57,199.57,7007.04,3,28,11.00,0.00
59029,2021-02-26,Stanislaus,California,55887,946.00,6099,CA,County,550660,10149.09,171.79,171.79,10149.09,4,10,112.00,2.00
73146,2021-02-26,Tulare,California,47869,758.00,6107,CA,County,466195,10268.02,162.59,162.59,10268.02,5,9,85.00,12.00
27417,2021-02-26,Riverside,California,289450,3767.00,6065,CA,County,2470546,11716.03,152.48,152.48,11716.03,6,6,265.00,18.00
46995,2021-02-26,San Joaquin,California,66709,1101.00,6077,CA,County,762148,8752.76,144.46,144.46,8752.76,7,18,140.00,19.00
418585,2021-02-26,Merced,California,28971,397.00,6047,CA,County,277680,10433.23,142.97,142.97,10433.23,8,8,56.00,2.00
559580,2021-02-26,Kings,California,22003,218.00,6031,CA,County,152940,14386.69,142.54,142.54,14386.69,9,3,52.00,0.00
26703,2021-02-26,Fresno,California,95015,1422.00,6019,CA,County,999101,9510.05,142.33,142.33,9510.05,10,14,221.00,10.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
972628,2021-02-26,Lassen,California,5601,19.00,6035,CA,County,30573,18320.09,62.15,62.15,18320.09,37,1,0.00,0.00
264071,2021-02-26,Imperial,California,26855,586.00,6025,CA,County,181215,14819.41,323.37,323.37,14819.41,1,2,12.00,1.00
559580,2021-02-26,Kings,California,22003,218.00,6031,CA,County,152940,14386.69,142.54,142.54,14386.69,9,3,52.00,0.00
131739,2021-02-26,San Bernardino,California,286077,2783.00,6071,CA,County,2180085,13122.29,127.66,127.66,13122.29,12,4,743.00,110.00
1997,2021-02-26,Los Angeles,California,1189859,21241.00,6037,CA,County,10039107,11852.24,211.58,211.58,11852.24,2,5,1758.00,139.00
27417,2021-02-26,Riverside,California,289450,3767.00,6065,CA,County,2470546,11716.03,152.48,152.48,11716.03,6,6,265.00,18.00
165537,2021-02-26,Kern,California,102998,826.00,6029,CA,County,900202,11441.65,91.76,91.76,11441.65,24,7,371.00,15.00
418585,2021-02-26,Merced,California,28971,397.00,6047,CA,County,277680,10433.23,142.97,142.97,10433.23,8,8,56.00,2.00
73146,2021-02-26,Tulare,California,47869,758.00,6107,CA,County,466195,10268.02,162.59,162.59,10268.02,5,9,85.00,12.00
59029,2021-02-26,Stanislaus,California,55887,946.00,6099,CA,County,550660,10149.09,171.79,171.79,10149.09,4,10,112.00,2.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
264071,2021-02-26,Imperial,California,26855,586.00,6025,CA,County,181215,14819.41,323.37,323.37,14819.41,1,2,12.00,1.00
1997,2021-02-26,Los Angeles,California,1189859,21241.00,6037,CA,County,10039107,11852.24,211.58,211.58,11852.24,2,5,1758.00,139.00
461748,2021-02-26,Inyo,California,1264,36.00,6027,CA,County,18039,7007.04,199.57,199.57,7007.04,3,28,11.00,0.00
59029,2021-02-26,Stanislaus,California,55887,946.00,6099,CA,County,550660,10149.09,171.79,171.79,10149.09,4,10,112.00,2.00
73146,2021-02-26,Tulare,California,47869,758.00,6107,CA,County,466195,10268.02,162.59,162.59,10268.02,5,9,85.00,12.00
27417,2021-02-26,Riverside,California,289450,3767.00,6065,CA,County,2470546,11716.03,152.48,152.48,11716.03,6,6,265.00,18.00
46995,2021-02-26,San Joaquin,California,66709,1101.00,6077,CA,County,762148,8752.76,144.46,144.46,8752.76,7,18,140.00,19.00
418585,2021-02-26,Merced,California,28971,397.00,6047,CA,County,277680,10433.23,142.97,142.97,10433.23,8,8,56.00,2.00
559580,2021-02-26,Kings,California,22003,218.00,6031,CA,County,152940,14386.69,142.54,142.54,14386.69,9,3,52.00,0.00
26703,2021-02-26,Fresno,California,95015,1422.00,6019,CA,County,999101,9510.05,142.33,142.33,9510.05,10,14,221.00,10.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,323.37,14819.41,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,323.37,14819.41,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,323.37,14819.41,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,323.37,14819.41,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,323.37,14819.41,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3452,02/22/21,Lassen,5591,19.00,18287.38,62.15,62.15,18320.09,37,1,33.00,0.00
3453,02/23/21,Lassen,5594,19.00,18297.19,62.15,62.15,18320.09,37,1,3.00,0.00
3454,02/24/21,Lassen,5594,19.00,18297.19,62.15,62.15,18320.09,37,1,0.00,0.00
3455,02/25/21,Lassen,5601,19.00,18320.09,62.15,62.15,18320.09,37,1,7.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,323.37,14819.41,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,323.37,14819.41,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,323.37,14819.41,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,323.37,14819.41,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,323.37,14819.41,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3526,02/22/21,Fresno,94374,1367.00,9445.89,136.82,142.33,9510.05,10,14,140.00,0.00
3527,02/23/21,Fresno,94584,1397.00,9466.91,139.83,142.33,9510.05,10,14,210.00,30.00
3528,02/24/21,Fresno,94741,1401.00,9482.62,140.23,142.33,9510.05,10,14,157.00,4.00
3529,02/25/21,Fresno,94794,1412.00,9487.93,141.33,142.33,9510.05,10,14,53.00,11.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)